# Local OLLAMA model test with LangChain

In [1]:
# set the root folder as ../
import os

os.chdir("../")
# get current working directory
print("Current working directory:", os.getcwd())

Current working directory: /home/ettore/projects/hackathons/MVA_hackathon_2025/vintage_ai


In [4]:
"""
LangGraph + Crawl4AI agent that
1) searches for the Ollama release-notes page,
2) crawls that URL,
3) reads the first bullet list,
4) returns the headline feature.

Fits on a 4 GB GTX-1060.
"""

# ---------------------------------------------------------------
# one-time shell installs (already done once = skip next time)
# ---------------------------------------------------------------
# pip install -U langchain langgraph langchain-ollama crawl4ai playwright
# crawl4ai-setup

# ---------------------------------------------------------------
# 0. imports
# ---------------------------------------------------------------
import asyncio

from crawl4ai import AsyncWebCrawler
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama.chat_models import ChatOllama
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field

# ---------------------------------------------------------------
# 1. tools
# ---------------------------------------------------------------
# 1-A  DuckDuckGo search (no schema needed; LangChain supplies one)
search_tool = DuckDuckGoSearchRun()

# 1-B  Crawl4AI fetch  -----------------------------------------
async_crawler = AsyncWebCrawler()


def crawl_page(url: str) -> str:
    """Return the main content of a page as Markdown."""
    return asyncio.run(async_crawler.arun(url=url)).markdown


class UrlSchema(BaseModel):
    url: str = Field(
        description="A fully-qualified link that starts with http:// or https://"
    )


crawl_tool = Tool(
    name="crawl_page",
    func=crawl_page,
    description="Fetch a web page and return its cleaned Markdown.",
    args_schema=UrlSchema,  # <-- forces the arg name ‘url’
)

# ---------------------------------------------------------------
# 2. LLM (tiny, tool-enabled)
#     – swap model="llama3.2:1b" if you insist on Meta’s build
# ---------------------------------------------------------------
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0.2,
    format="json",  # make Ollama speak OpenAI function-call JSON
).bind_tools(
    [search_tool, crawl_tool]
)  # attach schemas

# ---------------------------------------------------------------
# 3. ReAct graph
# ---------------------------------------------------------------
graph = create_react_agent(model=llm, tools=[search_tool, crawl_tool])

# ---------------------------------------------------------------
# 4. ask the question
# ---------------------------------------------------------------
question = (
    "Find who is the president of the United States right now and return the name. "
    "If you cannot find the answer, return 'I don't know'."
)

result = graph.invoke({"messages": [("user", question)]})

print("\n=== Final answer ===")
print(result["messages"][-1].content)


=== Final answer ===
{
    "title": "List of Presidents of the United States",
    "description": "A comprehensive list of all the presidents of the United States from 1789 to the present, with their portraits, terms, parties, and vice presidents.",
    "result": [
        {
            "name": "George Washington",
            "portrait": "Washington Portrait.jpg",
            "terms": "1789-1797",
            "party": "Federalist",
            "vicePresident": "John Adams"
        },
        {
            "name": "John Adams",
            "portrait": "Adams Portrait.jpg",
            "terms": "1797-1801",
            "party": "Federalist",
            "vicePresident": "Thomas Jefferson"
        },
        {
            "name": "Thomas Jefferson",
            "portrait": "Jefferson Portrait.jpg",
            "terms": "1801-1809",
            "party": "Democratic-Republican",
            "vicePresident": "Aaron Burr"
        },
        {
            "name": "James Madison",
           

In [ ]:
"""
insert_docs.py
--------------
Command-line utility to crawl any URL using Crawl4AI, detect content type (sitemap, .txt, or regular page),
use the appropriate crawl method, chunk the resulting Markdown into <1000 character blocks by header hierarchy,
and insert all chunks into ChromaDB with metadata.

Usage:
    python insert_docs.py <URL> [--collection ...] [--db-dir ...] [--embedding-model ...]
"""
import asyncio
import re
import sys
from types import SimpleNamespace
from typing import Any, Dict, List
from urllib.parse import urldefrag, urlparse
from xml.etree import ElementTree

import requests
from crawl4ai import (
    AsyncWebCrawler,
    BrowserConfig,
    CacheMode,
    CrawlerRunConfig,
    MemoryAdaptiveDispatcher,
)


def smart_chunk_markdown(markdown: str, max_len: int = 1000) -> List[str]:
    """Hierarchically splits markdown by #, ##, ### headers, then by characters, to ensure all chunks < max_len."""

    def split_by_header(md, header_pattern):
        indices = [m.start() for m in re.finditer(header_pattern, md, re.MULTILINE)]
        indices.append(len(md))
        return [
            md[indices[i] : indices[i + 1]].strip()
            for i in range(len(indices) - 1)
            if md[indices[i] : indices[i + 1]].strip()
        ]

    chunks = []

    for h1 in split_by_header(markdown, r"^# .+$"):
        if len(h1) > max_len:
            for h2 in split_by_header(h1, r"^## .+$"):
                if len(h2) > max_len:
                    for h3 in split_by_header(h2, r"^### .+$"):
                        if len(h3) > max_len:
                            for i in range(0, len(h3), max_len):
                                chunks.append(h3[i : i + max_len].strip())
                        else:
                            chunks.append(h3)
                else:
                    chunks.append(h2)
        else:
            chunks.append(h1)

    final_chunks = []

    for c in chunks:
        if len(c) > max_len:
            final_chunks.extend(
                [c[i : i + max_len].strip() for i in range(0, len(c), max_len)]
            )
        else:
            final_chunks.append(c)

    return [c for c in final_chunks if c]


def is_sitemap(url: str) -> bool:
    return url.endswith("sitemap.xml") or "sitemap" in urlparse(url).path


def is_txt(url: str) -> bool:
    return url.endswith(".txt")


async def crawl_recursive_internal_links(
    start_urls, max_depth=3, max_concurrent=10
) -> List[Dict[str, Any]]:
    """Recursive crawl using logic from 5-crawl_recursive_internal_links.py. Returns list of dicts with url and markdown."""
    browser_config = BrowserConfig(headless=True, verbose=False)
    run_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS, stream=False)
    dispatcher = MemoryAdaptiveDispatcher(
        memory_threshold_percent=70.0,
        check_interval=1.0,
        max_session_permit=max_concurrent,
    )

    visited = set()

    def normalize_url(url):
        return urldefrag(url)[0]

    current_urls = {normalize_url(u) for u in start_urls}
    results_all = []

    async with AsyncWebCrawler(config=browser_config) as crawler:
        for depth in range(max_depth):
            urls_to_crawl = [
                normalize_url(url)
                for url in current_urls
                if normalize_url(url) not in visited
            ]
            if not urls_to_crawl:
                break

            results = await crawler.arun_many(
                urls=urls_to_crawl, config=run_config, dispatcher=dispatcher
            )
            next_level_urls = set()

            for result in results:
                norm_url = normalize_url(result.url)
                visited.add(norm_url)

                if result.success and result.markdown:
                    results_all.append({"url": result.url, "markdown": result.markdown})
                    for link in result.links.get("internal", []):
                        next_url = normalize_url(link["href"])
                        if next_url not in visited:
                            next_level_urls.add(next_url)

            current_urls = next_level_urls

    return results_all


async def crawl_markdown_file(url: str) -> List[Dict[str, Any]]:
    """Crawl a .txt or markdown file using logic from 4-crawl_and_chunk_markdown.py."""
    browser_config = BrowserConfig(headless=True)
    crawl_config = CrawlerRunConfig()

    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(url=url, config=crawl_config)
        if result.success and result.markdown:
            return [{"url": url, "markdown": result.markdown}]
        else:
            print(f"Failed to crawl {url}: {result.error_message}")
            return []


def parse_sitemap(sitemap_url: str) -> List[str]:
    resp = requests.get(sitemap_url)
    urls = []

    if resp.status_code == 200:
        try:
            tree = ElementTree.fromstring(resp.content)
            urls = [loc.text for loc in tree.findall(".//{*}loc")]
        except Exception as e:
            print(f"Error parsing sitemap XML: {e}")

    return urls


async def crawl_batch(
    urls: List[str], max_concurrent: int = 10
) -> List[Dict[str, Any]]:
    """Batch crawl using logic from 3-crawl_sitemap_in_parallel.py."""
    browser_config = BrowserConfig(headless=True, verbose=False)
    crawl_config = CrawlerRunConfig(cache_mode=CacheMode.BYPASS, stream=False)
    dispatcher = MemoryAdaptiveDispatcher(
        memory_threshold_percent=70.0,
        check_interval=1.0,
        max_session_permit=max_concurrent,
    )

    async with AsyncWebCrawler(config=browser_config) as crawler:
        results = await crawler.arun_many(
            urls=urls, config=crawl_config, dispatcher=dispatcher
        )
        return [
            {"url": r.url, "markdown": r.markdown}
            for r in results
            if r.success and r.markdown
        ]


def extract_section_info(chunk: str) -> Dict[str, Any]:
    """Extracts headers and stats from a chunk."""
    headers = re.findall(r"^(#+)\s+(.+)$", chunk, re.MULTILINE)
    header_str = "; ".join([f"{h[0]} {h[1]}" for h in headers]) if headers else ""

    return {
        "headers": header_str,
        "char_count": len(chunk),
        "word_count": len(chunk.split()),
    }


def crawl(url: str):
    """Main function to crawl a URL, chunk the content, and prepare for db insertion."""

    args = SimpleNamespace()
    args.chunk_size = 1000
    args.max_depth = 3
    args.max_concurrent = 10
    args.batch_size = 100

    if is_txt(url):
        print(f"Detected .txt/markdown file: {url}")
        crawl_results = asyncio.run(crawl_markdown_file(url))
    elif is_sitemap(url):
        print(f"Detected sitemap: {url}")
        sitemap_urls = parse_sitemap(url)
        if not sitemap_urls:
            print("No URLs found in sitemap.")
            sys.exit(1)
        crawl_results = asyncio.run(
            crawl_batch(sitemap_urls, max_concurrent=args.max_concurrent)
        )
    else:
        print(f"Detected regular URL: {url}")
        crawl_results = asyncio.run(
            crawl_recursive_internal_links(
                [url], max_depth=args.max_depth, max_concurrent=args.max_concurrent
            )
        )

    # Chunk and collect metadata
    ids, documents, metadatas = [], [], []
    chunk_idx = 0
    for doc in crawl_results:
        url = doc["url"]
        md = doc["markdown"]
        chunks = smart_chunk_markdown(md, max_len=args.chunk_size)
        for chunk in chunks:
            ids.append(f"chunk-{chunk_idx}")
            documents.append(chunk)
            meta = extract_section_info(chunk)
            meta["chunk_index"] = chunk_idx
            meta["source"] = url
            metadatas.append(meta)
            chunk_idx += 1

    if not documents:
        print("No documents found to insert.")
        sys.exit(1)

    print(f"Found {len(documents)} chunks")

    print(f"Examople chunk metadata: {metadatas[0] if metadatas else 'N/A'}")
    print(f"Example chunk: {documents[0] if documents else 'N/A'}")

In [3]:
crawl("https://langchain-ai.github.io/langgraph/")  # Change URL as needed

Detected regular URL: https://langchain-ai.github.io/langgraph/


RuntimeError: asyncio.run() cannot be called from a running event loop